In [2]:
from pathlib import Path

import pandas as pd 
import numpy as np
import xarray as xr
import geopandas as gpd 
from shapely import box
import pycountry

from tqdm.notebook import tqdm

import config.config as cfg 
from config.config import (PROCESSED_GMT_FILE,
                           GRAPHICS_DIR, GRAPHICS_PAPER_MAIN_DIR, GRAPHICS_PAPER_APPENDIX_DIR, 
                           RESULTS_DIR,
                           fontsize_medium,
                           fontsize_small,
                           fontsize_large, 
                           regions,
                           region_naming_dict, 
                           population_shares, 
                           group_colors,
                           uncertainty_color,
                           sensitvitiy_color,
                           symbols,
                           EU27_ids
                           )
from utils.plotting_functions import lighten_color

import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import colormaps as cmaps
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, to_rgba
import matplotlib.colors as cm

from tabulate import tabulate

from itertools import product

plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
tas_frequency_df = xr.open_dataset(RESULTS_DIR / 'tas_frequency_hot.nc').stack(coords = ('lat', 'lon')).dropna('coords', how = 'all')
spei_frequency_df = xr.open_dataset(RESULTS_DIR / 'spei_frequency_dry.nc').stack(coords = ('lat', 'lon')).dropna('coords', how = 'all')
tas_intensity_df = xr.open_dataset(RESULTS_DIR / 'tas_intensity_hot.nc').stack(coords = ('lat', 'lon')).dropna('coords', how = 'all')
spei_intensity_df = xr.open_dataset(RESULTS_DIR / 'spei_intensity_dry.nc').stack(coords = ('lat', 'lon')).dropna('coords', how = 'all')

In [8]:
tas_frequency_df = xr.open_dataset(RESULTS_DIR / 'tas_frequency_hot.nc')
tas_frequency_df['attributed_tas'] = tas_frequency_df['attributed_tas'].assign_attrs({'description': 'Number of additonal heat events over 100 years attributable to the emissions of specific emitter groups', 
                                                                                      'units': 'days per 100 years'})
tas_frequency_df['1990_tas'] = tas_frequency_df['1990_tas'].assign_attrs({'description': 'Number of additonal heat events over 100 years attributable to all cumulative emissions since 1990', 
                                                                                      'units': 'days per 100 years'})
tas_frequency_df['region'] = tas_frequency_df['region'].assign_attrs({'description': 'Emitter regions (IN: India, CN: China)'})
tas_frequency_df['group'] = tas_frequency_df['group'].assign_attrs({'description': 'Income groups (p90p100: top 10, p99p100: top 1)'})
tas_frequency_df['model'] = tas_frequency_df['model'].assign_attrs({'description': 'CMIP6 model configuration from MESMER-M-TP'})
tas_frequency_df['month'] = tas_frequency_df['month'].assign_attrs({'description': 'Impacts on heat events in specific months (0: January, 11: December)'})
tas_frequency_df['extreme_quantile'] = tas_frequency_df['extreme_quantile'] .assign_attrs({'description': 'Event definition. (0.98: 1-in-50 year event, 0.99: 1-in-100 year event, 0.9999: 1-in-10000 year event)'})

tas_frequency_df.to_netcdf(RESULTS_DIR / 'Publication' / 'Extremes' / 'tas_frequency_hot.nc')

In [10]:
spei_frequency_df = xr.open_dataset(RESULTS_DIR / 'spei_frequency_dry.nc')
spei_frequency_df['attributed_spei'] = spei_frequency_df['attributed_spei'].assign_attrs({'description': 'Number of additonal drought events over 100 years attributable to the emissions of specific emitter groups', 
                                                                                      'units': 'days per 100 years'})
spei_frequency_df['1990_spei'] = spei_frequency_df['1990_spei'].assign_attrs({'description': 'Number of additonal drought events over 100 years attributable to all cumulative emissions since 1990', 
                                                                                      'units': 'days per 100 years'})
spei_frequency_df['region'] = spei_frequency_df['region'].assign_attrs({'description': 'Emitter regions (IN: India, CN: China)'})
spei_frequency_df['group'] = spei_frequency_df['group'].assign_attrs({'description': 'Income groups (p90p100: top 10, p99p100: top 1)'})
spei_frequency_df['model'] = spei_frequency_df['model'].assign_attrs({'description': 'CMIP6 model configuration from MESMER-M-TP'})
spei_frequency_df['month'] = spei_frequency_df['month'].assign_attrs({'description': 'Impacts on drought events in specific months (0: January, 11: December)'})
spei_frequency_df['extreme_quantile'] = spei_frequency_df['extreme_quantile'] .assign_attrs({'description': 'Event definition. (0.02: 1-in-50 year event, 0.01: 1-in-100 year event, 0.0001: 1-in-10000 year event)'})

spei_frequency_df.to_netcdf(RESULTS_DIR / 'Publication' / 'Extremes' / 'spei_frequency_dry.nc')

In [11]:
tas_intensity_df = xr.open_dataset(RESULTS_DIR / 'tas_intensity_hot.nc')
tas_intensity_df['attributed_tas'] = tas_intensity_df['attributed_tas'].assign_attrs({'description': 'Intensity increase of heat events attributable to the emissions of specific emitter groups', 
                                                                                      'units': '°C'})
tas_intensity_df['1990_tas'] = tas_intensity_df['1990_tas'].assign_attrs({'description': 'Intensity increase of heat events attributable to all cumulative emissions since 1990', 
                                                                                      'units': '°C'})
tas_intensity_df['region'] = tas_intensity_df['region'].assign_attrs({'description': 'Emitter regions (IN: India, CN: China)'})
tas_intensity_df['group'] = tas_intensity_df['group'].assign_attrs({'description': 'Income groups (p90p100: top 10, p99p100: top 1)'})
tas_intensity_df['model'] = tas_intensity_df['model'].assign_attrs({'description': 'CMIP6 model configuration from MESMER-M-TP'})
tas_intensity_df['month'] = tas_intensity_df['month'].assign_attrs({'description': 'Impacts on heat events in specific months (0: January, 11: December)'})
tas_intensity_df['extreme_quantile'] = tas_intensity_df['extreme_quantile'] .assign_attrs({'description': 'Event definition. (0.98: 1-in-50 year event, 0.99: 1-in-100 year event, 0.9999: 1-in-10000 year event)'})

tas_intensity_df.to_netcdf(RESULTS_DIR / 'Publication' / 'Extremes' / 'tas_intensity_hot.nc')

In [13]:
spei_intensity_df = xr.open_dataset(RESULTS_DIR / 'spei_intensity_dry.nc')
spei_intensity_df['attributed_spei'] = spei_intensity_df['attributed_spei'].assign_attrs({'description': 'Intensity increase of drought events attributable to the emissions of specific emitter groups', 
                                                                                      'units': '°C'})
spei_intensity_df['1990_spei'] = spei_intensity_df['1990_spei'].assign_attrs({'description': 'Intensity increase of heat events attributable to all cumulative emissions since 1990', 
                                                                                      'units': '°C'})
spei_intensity_df['region'] = spei_intensity_df['region'].assign_attrs({'description': 'Emitter regions (IN: India, CN: China)'})
spei_intensity_df['group'] = spei_intensity_df['group'].assign_attrs({'description': 'Income groups (p90p100: top 10, p99p100: top 1)'})
spei_intensity_df['model'] = spei_intensity_df['model'].assign_attrs({'description': 'CMIP6 model configuration from MESMER-M-TP'})
spei_intensity_df['month'] = spei_intensity_df['month'].assign_attrs({'description': 'Impacts on drought events in specific months (0: January, 11: December)'})
spei_intensity_df['extreme_quantile'] = spei_intensity_df['extreme_quantile'].assign_attrs({'description': 'Event definition. (0.02: 1-in-50 year event, 0.01: 1-in-100 year event, 0.0001: 1-in-10000 year event)'})

spei_intensity_df.to_netcdf(RESULTS_DIR / 'Publication' / 'Extremes' / 'spei_intensity_dry.nc')